In [2]:
import spacy
import json

## Train CPU: NER + morphologizer

#### Download base model

In [ ]:
# !python -m spacy download es_core_news_lg

#### Train model

In [ ]:
! python -m spacy train config_NER-MORF.cfg --output /src/ia2/ia2/models/ner_mor/  --paths.train /resources/datasets/docbin/train.spacy --paths.dev /resources/datasets/docbin/val.spacy

## Load Best Model

In [3]:
nlp_ner = spacy.load('/src/ia2/ia2/models/ner_mor/model-best')

In [4]:
nlp_ner.get_pipe('ner').labels

('ARTÍCULO', 'DIRECCIÓN', 'LOC', 'PER')

In [5]:
nlp_ner.pipeline

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x7fe658e0ee60>),
 ('morphologizer',
  <spacy.pipeline.morphologizer.Morphologizer at 0x7fe658e0f1c0>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x7fe65b1c3ae0>)]

## Evaluate

In [6]:
! python -m spacy evaluate /src/ia2/ia2/models/ner_mor/model-best  /resources/datasets/docbin/test.spacy --output /resources/ouputs/metrics/ner_mor.json --gold-preproc

ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

================================== Results ==================================

TOK     100.00
POS     -     
MORPH   -     
NER P   66.03 
NER R   81.05 
NER F   72.77 
SPEED   6146  


=============================== NER (per type) ===============================

                P       R       F
ARTÍCULO    70.46   86.09   77.50
LOC         67.66   81.29   73.86
DIRECCIÓN   72.93   88.99   80.17
PER         50.63   64.02   56.54

✔ Saved results to /resources/ouputs/metrics/ner_mor.json
